In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
import contractions
import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
data = pd.read_csv("stock_data.csv")

In [40]:
data

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [41]:
txt = list(data["Text"])
txt

['Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts',
 'user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  ',
 "user I'd be afraid to short AMZN - they are looking like a near-monopoly in eBooks and infrastructure-as-a-service",
 'MNTA Over 12.00  ',
 'OI  Over 21.37  ',
 'PGNX  Over 3.04  ',
 'AAP - user if so then the current downtrend will break. Otherwise just a short-term correction in med-term downtrend.',
 "Monday's relative weakness. NYX WIN TIE TAP ICE INT BMC AON C CHK BIIB  ",
 'GOOG - ower trend line channel test & volume support.   ',
 'AAP will watch tomorrow for ONG entry.',
 "i'm assuming FCX opens tomorrow above the 34.25 trigger buy. still very much like this setup.  ",
 'It really worries me how everyone expects the market to rally now,usually exact opposite happens every time we shall see soon bac spx jpm',
 "AAP GAMCO's arry Haverty : Apple Is Extremely Cheap  Great Video !!!"

In [42]:
stopwords = nltk.corpus.stopwords.words("english")

In [73]:
# spacy_model = spacy.load("en_core_web_sm", disable=["parser", "ner"])
spacy_model = spacy.load("en_core_web_sm")

In [74]:
def clean_text(txt):
    cleaned = []
    for row in txt:
        row = row.split(" ")
        # retain alphanumeric characters with contractions expanded
        row = [contractions.fix(w) for w in row]
        row = [w for w in row if w.isalnum() and w not in stopwords] 
        row = " ".join(row)
        row = " ".join([token.lemma_.lower() for token in spacy_model(row)])
        cleaned.append(row)
    return cleaned

In [75]:
cleaned_txt = clean_text(txt)

In [76]:
cleaned_txt

['kicker watchlist xide tit soq pnk cpw bpz aj trade method 1 method see prev post',
 'aap return indicator 15 trade',
 'user afraid short amzn look like ebooks',
 'mnta over',
 'oi over',
 'pgnx over',
 'aap user current downtrend otherwise correction',
 'relative nyx win tie tap ice int bmc aon c chk biib',
 'goog ower trend line channel test volume',
 'aap watch tomorrow ong',
 'assume fcx open tomorrow trigger still much like',
 'it really worry everyone expect market rally exact opposite happen every time shall see soon bac spx jpm',
 'aap arry haverty apple be extremely cheap great video',
 'user maykiljil post i agree msft go high possibly north 30',
 'momentum come back etfc break ma200 resistance solid volume ong',
 'ha hitting mean resume target 42 level',
 'user gameplan shoot today i like trend break may oc weekly trend break back july 2011',
 'fcx gapping well ideal entry look pull least 35 open entry',
 'user great list particularly like fisv all buy hold type check free 

In [77]:
# take a look at the k most common words in the entire dataset (and their respective freq)
k = 25
nltk_txt = nltk.Text(" ".join(cleaned_txt).split(" "))
nltk_vocab = nltk_txt.vocab()  
nltk_vocab.tabulate(k)

   aap      i  short   look  stock    buy   user   like     go    day  break   high   stop   long volume   good  watch   move   goog market    bac    get    new  still    see 
   884    458    433    314    302    297    295    280    280    274    269    268    248    244    243    227    225    222    206    206    204    201    201    196    192 


In [78]:
len(nltk_vocab)
# number of unique words in entire dataset

6691

In [79]:
from sklearn.model_selection import train_test_split
seed = 123
X = cleaned_txt
y = data["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=seed)

## SKlearn Classifier

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# count_vect = CountVectorizer() # can set n-gram range for multiple words to be grouped together
# X_train_counts = count_vect.fit_transform(X_train)
# # print(X_train_counts.shape)  # (number of sentences, number of unique words) = (4343, 5740)

# # TF-IDF
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# # print(X_train_tfidf.shape)  # (4343, 5740)

In [81]:
def sklearn_classifier_trial(clf, X_train, y_train, X_test, y_test):

    model = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', clf())])
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("Classifier: ", str(clf))
    print("Accuracy: ", np.mean(y_pred == y_test))
#     for e in range(len(y_test)):
#         print(y_pred[e], y_test.to_list()[e])
    print()
    return y_pred

In [84]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

clfs = [MultinomialNB, DecisionTreeClassifier, KNeighborsClassifier, RandomForestClassifier, LogisticRegression]

for clf in clfs:
    sklearn_classifier_trial(clf, X_train, y_train, X_test, y_test)

Classifier:  <class 'sklearn.naive_bayes.MultinomialNB'>
Accuracy:  0.7610008628127696

Classifier:  <class 'sklearn.tree._classes.DecisionTreeClassifier'>
Accuracy:  0.7428817946505608

Classifier:  <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Accuracy:  0.7402933563416738

Classifier:  <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Accuracy:  0.7842968075927523

Classifier:  <class 'sklearn.linear_model._logistic.LogisticRegression'>
Accuracy:  0.7842968075927523

